In [1]:
import pandas as pd
from collections import Counter
from PIL import Image,ImageDraw,ImageFont
from scipy.stats import spearmanr
import os
import numpy as np

In [2]:
def featparse(df):
    df['record_type'] = [item.split("_")[0] for item in df.feature]
    df['unit'] = [item.split("_")[1] for item in df.feature]
    df['num_agg'] = [item.split("_")[-1] for item in df.feature]

    feat_var = []
    temp_agg = []
    
    for i in df.index:
        featstr = df.feature.loc[i]
        if df.record_type.loc[i]=='ocnd':
            feat_var.append(featstr.split("_")[2])
            temp_agg.append(None)
        else:
            feat_var.append(featstr.split("_")[3])
            temp_agg.append(featstr.split("_")[2])
    
    df['feat_var'] = feat_var
    df['temp_agg'] = temp_agg

In [3]:
df = pd.read_csv("lime-robustness-all-score-outcome.csv")

In [4]:
feats = []
for i in df.index:
    tmp = df.unq.loc[i]
    tmp = tmp.rstrip("}")
    tmp = tmp.lstrip("{")
    feats.append(tmp.split(","))
feats = [item for sublist in feats for item in sublist]
feats = [item.translate(None," '") for item in feats]

In [5]:
limefeat = pd.DataFrame({"feature":[item[0] for item in Counter(feats).most_common()],
              "frequency":[item[1] for item in Counter(feats).most_common()]})

logregfeat = pd.read_csv("32092_2014-04-11_feature-importances.csv")
logregfeat = logregfeat[['feature','feature_importance']]
logregfeat['feature_importance_abs'] = logregfeat.feature_importance.apply(abs)
logregfeat.sort_values(by="feature_importance_abs",ascending=False,inplace=True)
logregfeat.reset_index(drop=True,inplace=True)

In [6]:
featparse(limefeat)
featparse(logregfeat)

In [7]:
limefeatlist = set(limefeat.feature)
logregfeatlist = set(logregfeat.feature)
print len(limefeatlist),len(logregfeatlist),len(limefeatlist.intersection(logregfeatlist)),len(logregfeatlist-limefeatlist),1459-1276

1276 1459 1276 183 183


Adding the features LIME didn't pick up to the limefeat dataframe, so that we can do a direct comparison with the feature importances from the original scikit-learn logistic regression model.

In [8]:
tmp = pd.DataFrame({"feature":list(logregfeatlist-limefeatlist),
                    "frequency":0})
featparse(tmp)
limefeat = limefeat.append(tmp)
limefeat.reset_index(drop=True,inplace=True)

#### Sorting LIME table according to scikit-learn LogReg feature importance ordering

In [9]:
tmp = logregfeat[['feature']]
tmp.reset_index(drop=False,inplace=True)
tmp.set_index("feature",inplace=True)
tmp.rename(columns={"index":"idx"},inplace=True)
limefeat = limefeat.join(tmp,on="feature")

/Library/Python/2.7/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [10]:
limefeat.to_csv("limefeat.csv",index=False)
logregfeat.to_csv("logregfeat.csv",index=False)